# Classification de texte avec un MLP PyTorch sur Bag-of-Words

Ce notebook explique en détail comment entraîner un perceptron multicouche (MLP) avec PyTorch pour la classification de texte, à partir de représentations bag-of-words prétraitées. 
Chaque étape est expliquée, du chargement des données à la génération du fichier de soumission.

---

## Concepts clés

- **Bag-of-Words (BOW)** : Représentation vectorielle d'un texte où chaque dimension correspond à un mot du vocabulaire, et la valeur est le nombre d'occurrences du mot dans le texte.
- **MLP (Multi-Layer Perceptron)** : Réseau de neurones à couches entièrement connectées, capable de modéliser des relations non linéaires.
- **CrossEntropyLoss** : Fonction de perte adaptée à la classification multi-classes, qui mesure la différence entre la distribution prédite et la vraie classe.
- **Adam** : Optimiseur efficace pour l'entraînement des réseaux de neurones.

---

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
import joblib
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd

## Version et introduction

On affiche la version du script et on explique le but : entraîner un modèle de classification de texte avec PyTorch.

In [ ]:
print("V2_1.3.1 – Entraînement + Prédiction PyTorch corrigé")

## 1. Chargement des artefacts prétraités

On charge les matrices numpy contenant les représentations bag-of-words (BOW) des textes d'entraînement et de test, ainsi que les labels encodés en one-hot. 
On charge aussi le vectorizer et le label encoder pour décoder les prédictions.

In [ ]:
X_train = np.load('X_train.npy')        # représentation bag-of-words pad/trunc
y_train_onehot = np.load('y_train.npy') # one-hot
X_test = np.load('X_test.npy')          # représentation bag-of-words pad/trunc

vectorizer = joblib.load('vectorizer.joblib')     # transformateur BOW
le = joblib.load('label_encoder.joblib')           # encodeur de labels

## 2. Préparation des labels pour CrossEntropyLoss

La fonction de perte `CrossEntropyLoss` attend des indices de classes (entiers), pas des vecteurs one-hot. On convertit donc les labels one-hot en indices avec `np.argmax`.

In [ ]:
y_train = np.argmax(y_train_onehot, axis=1)

## 3. Conversion en tensors PyTorch

On convertit les matrices numpy en tensors PyTorch pour pouvoir les utiliser dans le DataLoader et le modèle. 
On crée un DataLoader pour itérer sur les données par mini-batchs lors de l'entraînement.

In [ ]:
dtype = torch.float32
X_tr = torch.tensor(X_train, dtype=dtype)
y_tr = torch.tensor(y_train, dtype=torch.long)
X_te = torch.tensor(X_test, dtype=dtype)

dataset = TensorDataset(X_tr, y_tr)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

## 4. Définition du modèle MLP pour BOW

On définit un perceptron multicouche (MLP) simple :
- Une couche linéaire (entrée → cachée)
- Activation ReLU (introduit de la non-linéarité)
- Dropout (régularisation pour éviter le surapprentissage)
- Une couche linéaire (cachée → sortie)

Mathématiquement, le MLP calcule :
$$ y = W_2 (\text{ReLU}(W_1 x + b_1)) + b_2 $$
où $x$ est le vecteur BOW, $W_1$, $W_2$ sont les matrices de poids, $b_1$, $b_2$ les biais.

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        return self.net(x)

input_dim = X_train.shape[1]  # longueur du vecteur BOW
hidden_dim = 64
num_classes = y_train_onehot.shape[1]
model = SimpleMLP(input_dim, hidden_dim, num_classes)

## 5. Critère et optimiseur

- **CrossEntropyLoss** : combine un softmax et la log-vraisemblance pour la classification multi-classes.
- **Adam** : optimiseur adaptatif efficace pour l'entraînement des réseaux de neurones.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

## 6. Entraînement du modèle

On entraîne le modèle pendant plusieurs époques (passes sur toutes les données). À chaque batch :
- On calcule les logits (sorties non normalisées du modèle)
- On calcule la perte
- On rétro-propage le gradient et on met à jour les poids

La perte moyenne par batch est affichée à chaque époque pour suivre la convergence.

In [ ]:
epochs = 15
for epoch in range(1, epochs+1):
    model.train()
    total_loss = 0.0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch}/{epochs} – Loss: {total_loss/len(train_loader):.4f}")

## 7. Prédiction sur les données de test

On met le modèle en mode évaluation, on calcule les logits pour chaque texte de test, puis on prend la classe avec la probabilité la plus élevée (`argmax`).
On utilise le label encoder pour retrouver les noms de catégories d'origine.

In [ ]:
model.eval()
with torch.no_grad():
    logits = model(X_te)
    preds = torch.argmax(logits, dim=1).cpu().numpy()
    pred_labels = le.inverse_transform(preds)

## 8. Génération du fichier de soumission

On prépare le fichier de soumission au format attendu, associant chaque Id de test à la catégorie prédite.

In [ ]:
print("Saving submission2_2.csv...")
# Charger les Id du test
test_ids = pd.read_json('test.json', orient='records')['Id']
submission = pd.DataFrame({
    'Id': test_ids,
    'Category': pred_labels
})
submission.to_csv('submission2_2.csv', index=False)
print("✅ submission2_2.csv généré.")

---

## Concepts mathématiques et conclusion

- **Bag-of-Words** : chaque texte est représenté par un vecteur de dimension $d$ (taille du vocabulaire ou tronquée), où chaque entrée $x_i$ est le nombre d'occurrences du mot $i$ dans le texte.
- **MLP** : le réseau apprend une fonction $f(x) = \text{softmax}(W_2 \cdot \text{ReLU}(W_1 x + b_1) + b_2)$ qui approxime la probabilité d'appartenance à chaque classe.
- **CrossEntropyLoss** : pour chaque exemple, la perte est $-\log(p_{y})$ où $p_{y}$ est la probabilité prédite pour la vraie classe.

Ce pipeline montre comment passer de textes bruts à des prédictions de classes avec un réseau de neurones, en utilisant des représentations vectorielles simples mais efficaces.